<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os
from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from requests import Session

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params=None):
    if query_params is None:
        query_params = {}

    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path=None, action="actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key=None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables=None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)
NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(NEW_RELIC_USER_KEY)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(
            response,
            action="actor",
        ),
        indent=4,
    )
)

## Playground area

### Doint NRQL with the API

In [ ]:
query_string = """
    {
        actor {
            nrql(
                query: "SELECT * FROM Transaction",
                accounts: %(account_id)d,
            ) {
                nrql
                results
            }
        }
    }
"""

query_params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)

In [ ]:
results_df = pd.DataFrame(
    data=results,
)

In [ ]:
results_df

In [ ]:
query_string = """
    {
        actor {
            nrql(
                query: "FROM Transaction SELECT average(duration) FACET appName SINCE 1 hour ago TIMESERIES 1 minute",
                accounts: %(account_id)d,
            ) {
                nrql
                results
            }
        }
    }
"""

query_params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)

In [ ]:
results_df = pd.DataFrame(
    data=results,
)

In [ ]:
results_df

In [ ]:
app_names = results_df["appName"].unique()

In [ ]:
app_names

In [ ]:
plt.clf()

fig = plt.figure(0, figsize=(10, 5))
axs = fig.add_subplot(1, 1, 1)

for app_name in app_names:
    data = results_df.query("appName == @app_name")
    data = data[["beginTimeSeconds", "average.duration"]]
    data["beginTimeSeconds"] = pd.to_datetime(
        data["beginTimeSeconds"],
        unit="s",
    )

    axs.plot(
        data["beginTimeSeconds"],
        data["average.duration"],
        label=app_name,
        marker=".",
    )

fig.legend(
    frameon=False,
    loc="lower center",
    ncols=3,
    bbox_to_anchor=(0.5, -0.1),
)

fig.tight_layout()

plt.show()

### It's your turn!

From now on you can use the client defined in the previous section to make your queries to the New Relic GraphQL API

In [ ]:
# Put your code here :D